# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Fedora VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs.


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")


In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
fedora_server_name = f"{exp_name}-fedora-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)


In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()


In [ ]:
ubuntu_image = "CC-Ubuntu24.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

ubuntu_vm.submit(idempotent=True)


In [ ]:
fedora_image = "CC-Fedora39"

fedora_vm = server.Server(
    name=fedora_server_name,
    image_name=fedora_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

fedora_vm.submit(idempotent=True)


In [ ]:
ubuntu_vm.associate_floating_ip()
fedora_vm.associate_floating_ip()

ubuntu_ip = ubuntu_vm.get_floating_ip()
fedora_ip = fedora_vm.get_floating_ip()

print("Ubuntu VM IP:", ubuntu_ip)
print("Fedora VM IP:", fedora_ip)


In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
fedora_vm.add_security_group(sg.id)


In [ ]:
ubuntu_vm.check_connectivity()
fedora_vm.check_connectivity()

print("Both Ubuntu and Fedora VMs are ready.")


In [ ]:
print("SSH into Ubuntu (Sciunit author):")
print(f"ssh cc@{ubuntu_ip}")

print("\nSSH into Fedora (Sciunit reproducer):")
print(f"ssh cc@{fedora_ip}")


## Cleanup 

In [ ]:
ubuntu_vm.delete()
fedora_vm.delete()
l.delete()

print("Cleanup commands prepared (commented out).")


_______________________________________________________________________________

## Environment Check and Setup

Before starting the Sciunit Hello World example, we verify that:

1. Sciunit is installed and accessible
2. The environment is ready for terminal-based experiments

This section ensures all required tools are present before proceeding.


### Ensure Sciunit is discoverable

Sciunit is installed in the user-local directory (`~/.local/bin`).
We explicitly add this directory to the PATH for the Jupyter environment.

and 

### Check Sciunit installation status

We check whether the `sciunit` command is already available.
If it is not installed, we will install it in the next step.

In [ ]:
import os, shutil

os.environ["PATH"] = os.path.expanduser("~/.local/bin") + ":" + os.environ["PATH"]

if shutil.which("sciunit") is None:
    print("Sciunit is NOT installed. Please run the next cell.")
else:
    print("Sciunit is already installed and available.")


Only run the below cells if sciunit is not already installed 

In [ ]:
import os, shutil, subprocess

os.environ["PATH"] = os.path.expanduser("~/.local/bin") + ":" + os.environ["PATH"]

if shutil.which("sciunit") is None:
    print("Installing Sciunit...")
    subprocess.run(["pip", "install", "--user", "sciunit2"], check=True)
else:
    print("Sciunit already installed.")

subprocess.run(["sciunit", "--version"], check=True)


At the end of output, you should see "sciunit2 0.4.post115.dev43584757" which tells sciunit is installed. Now open the terminal and then check "sciunit --version" which should show the same.

we use this terminal as user 1 who tries to create a sciunit container for a simple hello world program.

Lets open another terminal and create a user 2, so that we can repeat what the user 1 has created for reproducibility. Follow the instructions below. 


open another terminal(terminal2) and enter the command

"sudo adduser user2"


You will be prompted to assign a password for user2, you can enter the password and confirm it, then enter user info (you can press Enter to skip the info fields). Finally confirm by pressing “Y” when prompted.
This process will create a new user account with its own home directory (e.g. /home/user2).

use the command "sudo -i -u user2" to impersonate the user 2. you will see the shell prompt change to user2.

lets install sciunit here as well with the following commands

"pip install --user sciunit2"

once sciunit is installed, check for the same "sciunit2 0.4.post115.dev43584757" at the end.
next, run the following command.


In [ ]:
```bash
export PATH=$HOME/.local/bin:$PATH

now check the version using the following command

"sciunit --version"

this should show sciunit2 0.4.post115.dev43584757

Now, lets move to sciunit_demo_run jupyter file for learning how to create a sciunit with terminal1 (default user1) and reproduce it using terminal2 (user2) 